In [ ]:
import pandas as pd

# # 한국시간 적용
---

In [1]:
from datetime import date, datetime, timezone, timedelta

exp_day = str(date.today())

KST = timezone(timedelta(hours=9))
time_record = datetime.now(KST)
_day = str(time_record)[:10]
_time = str(time_record.time())[:8]

print(exp_day) 
print(datetime.now(KST))
print(_day)
print(_time)

2022-06-18
2022-06-18 18:40:47.454174+09:00
2022-06-18
18:40:47


In [7]:
datetime.now().timestamp()

1655545293.563067

In [5]:
datetime.now(KST).timestamp()

1655545283.942352

In [ ]:
(time_record - timedelta(hours=1)).time()

datetime.time(19, 21, 53, 283520)

In [ ]:
import re

re.sub('-', '', _day)

'20220602'

In [ ]:
time = str(time_record.time())[:5]
if int(time[3:5]) < 30:
    time = time[0:2] + '00'
else:
    time = time[0:2] + '30'
print(time)

2000


In [ ]:
time_record.time().minute

26

# # 기상청 단기예보
---

In [ ]:
Service_key = 'dDUc9SVR/pZIxs5qbu7ABEG4/S5t0ZJBTLZdlveVsR4Hyc2ynwn17cF68xglOrHN/mNu/W4JmkPP3N/M8HF12g==' # 서비스키 1
# Service_key = 'p6rQtYpFx5ZivDokXiWeZEPT1qg7CGrnWM9XP9rPcOy45BUhyI3RM9OoXCo3CWSAH31Va67TZGHaOOFWN/I7Cg==' # 서비스키 2

x = '58'
y = '125'

def weather_forecast(Service_key, x, y):
    import requests
    import re
    import numpy as np 
    import pandas as pd
    from datetime import date, datetime, timezone, timedelta

    KST = timezone(timedelta(hours=9))
    time_record = datetime.now(KST)
    day = str(time_record)[:10]

    if time_record.time().minute > 40:
        time = str((time_record).time())[:2] + '00' # 실황에 사용
    elif time_record.time().minute <= 40:
        time = str((time_record - timedelta(hours=1)).time())[:2] + '00' # 예보에 사용

    # url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst' # 단기예보
    # url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst' # 초단기예보
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst' # 초단기실황
    params = {'serviceKey': f'{Service_key}',
              'pageNO': '1',
              'numOfRows': '60',
              'dataType': 'json',
              'base_date': f"{re.sub('-', '', day)}",
              'base_time': f'{time}',
              'nx': f'{x}',
              'ny': f'{y}'}

    response = requests.get(url, params=params).json()
    weather = response

    weather_df = pd.DataFrame(data = weather['response']['body']['items']['item'], index = ['강수형태', '습도', '1시간 강수량', '온도', '동서바람풍속', '동서바람풍향', '남북바람풍속', '남북바람풍향'])
    weather_df.drop(['baseDate', 'baseTime', 'nx', 'ny', 'category'], axis = 1, inplace = True)
    weather_df = weather_df.T

    pty_code = {0: '없음', 1: '비', 2: '비/눈', 3: '눈', 5: '빗방울', 6: '빗방울눈날림', 7: '눈날림'}
    pty_num = int(weather_df['강수형태']['obsrValue'])
    weather_df['강수형태']['obsrValue'] = pty_code[pty_num]

    if float(weather_df['동서바람풍속']['obsrValue']) > 0:
        weather_df['동서바람풍향']['obsrValue'] = '서'
    elif float(weather_df['동서바람풍속']['obsrValue']) < 0:
        weather_df['동서바람풍향']['obsrValue'] = '동'
    elif float(weather_df['동서바람풍속']['obsrValue']) == 0:
        weather_df['동서바람풍향']['obsrValue'] = '무'

    if float(weather_df['남북바람풍속']['obsrValue']) > 0:
        weather_df['남북바람풍향']['obsrValue'] = '남'
    elif float(weather_df['남북바람풍속']['obsrValue']) < 0:
        weather_df['남북바람풍향']['obsrValue'] = '북'
    elif float(weather_df['남북바람풍속']['obsrValue']) == 0:
        weather_df['남북바람풍향']['obsrValue'] = '무'

    return weather_df

In [ ]:
weather_df = weather_forecast(Service_key, x, y)
weather_df

,강수형태,습도,1시간 강수량,온도,동서바람풍속,동서바람풍향,남북바람풍속,남북바람풍향
obsrValue,없음,44,0,29.1,2.2,서,2.3,남
